In [2]:
from google.colab import files
files.upload()

Saving reviews.csv to reviews.csv


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('reviews.csv')

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [0]:
import nltk
from nltk.tokenize import word_tokenize

In [0]:
from sklearn.linear_model import LogisticRegression

In [6]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.2MB/s 


In [0]:
!pip -qq install rusenttokenize --progress-bar off

In [0]:
import pymorphy2

In [0]:
from rusenttokenize import ru_sent_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
import string

In [0]:
def preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

In [17]:
vectorizer = CountVectorizer(tokenizer=preprocess_tokenize)
bow = vectorizer.fit_transform(df.text)
bow = vectorizer.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bow, df.label)
lr = LogisticRegression(C=7, penalty='l1', random_state=10,  solver = 'saga' )
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.63      0.60      0.61      1040
           0       0.39      0.27      0.32      1427
           1       0.89      0.94      0.92      9159

    accuracy                           0.83     11626
   macro avg       0.64      0.60      0.62     11626
weighted avg       0.81      0.83      0.82     11626



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


# Веса признаков

In [19]:
clf.coef_.shape

(3, 141369)

In [20]:
print(max(enumerate(clf.coef_[0]), key=lambda pair: pair[1]),
      max(enumerate(clf.coef_[1]), key=lambda pair: pair[1]),
      max(enumerate(clf.coef_[2]), key=lambda pair: pair[1]))

(125461, 0.47697246313828073) (69087, 0.9172212305785721) (91368, 0.4095882700655025)


In [0]:
index_to_word = {
    ind: word
    for (word, ind)
    in vectorizer.vocabulary_.items()
    }

In [24]:
print("Топ-слово для отрицательных отзывов: ", '"',
      index_to_word[125461], '"', "\n", 
      "Топ-слово для нейтральных отзывов: ", '"',
      index_to_word[69087], '"', "\n", 
      "Топ-слово для положительных отзывов: ", '"',
      index_to_word[91368], '"')

Топ-слово для отрицательных отзывов:  " унылый " 
 Топ-слово для нейтральных отзывов:  " нейтральный " 
 Топ-слово для положительных отзывов:  " потрясать "


In [25]:
# первые 10 коэффициентов для отрицательных отзывов
top_features = sorted(enumerate(clf.coef_[0]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

унылый
разочарование
скучный
бред
разочаровать
скучно
нелепо
обидный
единственный
пародия


In [26]:
# первые 10 коэффициентов для нейтральных отзывов
top_features = sorted(enumerate(clf.coef_[1]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

нейтральный
скучноватый
противоречивый
поверхностно
подростковый
оценивать
душевность
вставить
неплохой
чудный


In [27]:
# первые 10 коэффициентов для положительных отзывов
top_features = sorted(enumerate(clf.coef_[2]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

потрясать
увлекательный
задуматься
однозначно
потрясающе
дыхание
рекомендовать
замечательный
превзойти
наслаждаться
